# The Compound Poisson Process

In this exercise we are going to look at an example of a hidden Markov chain - the compound Poisson Process.    
Before you start working through the exercises watch this video that explains what a compound Poisson process is.  Then in your notes:

- Explain what a compound poisson process is.
- Reproduce the derivation that explains how the average total spent $\mathbb{E}[Y(t)]$ changes as a function of time 
- Reproduce the derivation that explains how the standard deviation for the total ammount changes $\textrm{var}[Y(t)]$ as a function time.

In addition to studying this material you will also need to revise what learnt about the Poisson process.  In particular you need to make sure you know what the average number of events, $\mathbb{E}[N(t)]$, that have occured by time $t$ is equal to and what the standard deviation, $\textrm{var}[N(t)]$ for this random variable is equal to.

Once you have revised all this material press shift and enter on the cell below and we will begin

In [ ]:
import math
%matplotlib notebook
import matplotlib
import matplotlib.pyplot as plt
import matplotlib.animation as anim
from matplotlib import rc
from IPython.display import HTML
import random
rc('animation', html='none')

class plotobj(object) :
    def __init__( self, ngen ):
        self.ngen = ngen 
        self.fig = plt.figure()
        self.ax = plt.axes(xlim=(0, self.ngen), ylim=(0, 0.1))
        
    def setup(self):
        self.xdata, self.ydata = [], []
        self.line, = self.ax.plot([],[],'.')
        return self.line,
    
    def run( self, data ):
        t,N = data
        self.xdata.append(t)
        self.ydata.append(N)
        self.ax.set_xlim( 0, self.xdata[-1] )
        self.ax.set_ylim( 0, self.ydata[-1] )
        self.line.set_data( self.xdata, self.ydata )
        return self.line,
    
def raw( ngen, tn, yn, myvar ):
    cnt = 0
    while cnt < ngen :
        cnt += 1
        tn, yn = myvar(tn,yn)
        yield tn, yn
    
def dynamicplot( ngen, myvar ):
    tn, yn, myplot = 0, 0, plotobj( ngen )
    video = anim.FuncAnimation(fig=myplot.fig, func=myplot.run, frames=raw( ngen, tn, yn, myvar ), 
                                init_func=myplot.setup, interval=100, blit=False, repeat=False)
    return video

# Simulating the Compound Poisson Process

The first thing we need to know for this exercise is how to generate random variable from a normal distribution using python.  This is quite easy as there is an intrinsic function for just this purpose.  We can thus generate a Normal  random variable from a distribution with mean 2 and variance 0.1 using a call like the one in the cell below:  

In [ ]:
random.gauss(2,0.1)

With that under our belt we now have everything we need in order to simulate a compound poisson process.  I would like you to use the cell below to write a function that returns the time of the next event, $T_{n+1}$, for your compound Poisson process and the value of the total ammount spent after the event has occured, $Y_{n+1}$.  The cell below contains a template for how this should be done. I have defined a fucntion that takes as input the time at which the previous event occured, $T_n$, and the ammount that had been spent by customers up to that point $Y_n$.  I have also provided you with a call to the dynamic plotting tool that I have written above so you can view how $Y$ (the total ammount spent) changes as a function of time.  

When you do this exercise set $\lambda=2$ (the rate) and $\mu=4$ (the mean) in your compound Poisson process.  <b> Once you have finished this exercise and you have generated a graph describe the shape of this graph, and the (approximate) value of the gradient of the curve in your notes</b>.  

In [ ]:
def gencompoundpoisson( Tn, Yn ):
    Tn1 = Tn + 1     # You need to change this line
    Yn1 = Yn + 1    # and this line 
    return Tn1, Yn1

dynamicplot( 100, gencompoundpoisson )

In [ ]:
# Use this cell to test your code before you try to run the animation as 
# errors will not be returned while the animated plot constructs
gencompoundpoisson( 3, 9 )

Copy the code from above into the cell below and modify the code so that it can be used to simulate a regular Poisson process.  Set $\lambda=8$ for your Poisson process.  Describe the shape of the curve that you obtain when you simulate the Poisson process and give an approximate value for the gradient of the curve.   

# Hidden Markov processes

The Compound Poisson Process is an example of Hidden Markov Process.  A hidden Markov process is like a Markov chain but now we receive a random signal when we arrive in each state rather than a signal that tells us that we are in state $n$.  We can understand this statement in terms of the Compound Poisson Process as follows.  Suppose we have just have a regular Poisson process and we monitor the number of events, $N(t)$, that have occured by time $t$.  For the Poisson process we know the state that we are in at all times. If $N(t)$ is equal to $K$ then there have been exactly $K$ events and we are thus in state $K$.  Now suppose instead that we have a Compound Poisson Process that models the ammount of money spent on amazon.com as a function of time.  By time $t$ there have been $N(t)$ events (i.e. $N(t)$ purchases).  We, however, do not know this as we do no know how many events have occured - we only have information on the total ammount spent $Y(t)$.  In other words, the signal (the ammount spent by each customer) that is given each time there is an event (a purchase) and a change of state is random.  The underlying Markov chain (the Poisson Process) is thus hidden.

Lets suppose that we now have a time series for a random process.  In other words, we have the values our random process takes at a set of discrete points in time - a (longer version) of a table of values something like this:

<table>
<tr>
<td width="40"> $t$ </td> <td> 0 </td> <td> 5 </td> <td> 10 </td> <td> 15 </td> <td> 20 </td>
</tr>
<tr>
<td> $Y(t)$ </td> <td> 0 </td> <td> 3 </td> <td> 15 </td> <td> 17 </td> <td> 25 </td>
</tr>
<table>

How would we determine whether the stochastic process underlying this random variable is a Poisson Process or a Compound Poisson Process?  In other words, should we assume this data came from a Poisson process or a Compound Poisson process when we fit it?  Can we even work out which is the appropriate distribution?

To answer this question lets return to our old tricks.  We will simulate a Poisson process for a fixed period of time.  This will generate a random variable $N_t$.   By repeating this exercise multiple times we will get an estimate for the mean of our random variable using the law of large numbers.  We can then do the same exercise for the compound Poisson process.  With this in mind press shift and enter on the cell below, which contains some tools that we are now familiar with.  These tools allow us to plot a graph showing how our esimtate of the mean changes as we perform more experiments (simulate more time windows in this case). 

In [ ]:
class plotobj2(object) :
    def __init__(self, ngen ) :
        self.ngen=ngen
        self.fig = plt.figure()
        self.ax = plt.axes(xlim=(0, self.ngen), ylim=(0, 1))
    
    def setup(self):
        self.xdata=[]
        self.ydata=[]
        self.line, = self.ax.plot([],[],'.')
        return self.line,
    
    def run(self,data):
        t,y = data
        self.xdata.append(t)
        self.ydata.append(y)
        self.ax.set_ylim( 0, 2*max(self.ydata) )
        self.line.set_data(self.xdata, self.ydata )
        return self.line,
        
def dynamicplot2( ngen, operation, myvar):
    myplot = plotobj2( ngen )
    operation.rsum=0
    return anim.FuncAnimation(fig=myplot.fig, func=myplot.run, frames=operation( ngen, myvar ), 
                                init_func=myplot.setup, interval=10, blit=False, repeat=False)

def mean( ngen, myvar ) :
    mysum = 0
    cnt = 0
    while cnt < ngen :
        cnt += 1
        mysum += myvar()
        yield cnt, mysum / cnt

Lets now use these tools to look at how the number of events that occur during a time window of length 10 seconds for a Poisson process with $\lambda=1$ (the rate in units of s$^{-1}$).  Most of the code that you will need to do this is in the cell follows.  You will need to write the function that simulates the Poisson process for 10 seconds - there is some guidance in the comments.  The cell below allows you to test your implementation of the function.  Once you are happy that your function works (it should not return errors) then use the next but one cell to use the tools above to plot the estimate of the mean as you generate more time windows.  

In [ ]:
# You will need to write the function below.  
# It should return a random number that tells 
# you how many events have occur during a time 
# window of length 10 seconds during a simulation of
# a Poisson process.  You have seen two ways of doing this 
# in this exercises and in previous exercises.
def poisson():
    return 1

poisson()

In [ ]:
dynamicplot2( 1000, mean, poisson )

Use the cell below to write a function that simulations a Compound Poisson Process for 10 seconds.  In this compound Poisson you should set $\lambda=1$ (the rate) and $\mu=1$ (the mean).  There are two ways that you can implement this function using ideas from previous exercises.  As above you can test the function by pressing shift and enter on the cell below.  In the second cell under this one by contrast you can call dynamic plot and thus see how the estimate of the mean changes as you generate more samples.   

In [ ]:
def compoundpoisson() :
    return 1

In [ ]:
dynamicplot2( 1000, mean, compoundpoisson )

You will find that the estimates of the mean for the Poisson process is the same as the estimate of the mean for the Compound Poisson process.  <b> In your notes explain why this is the case by making reference to the theory of the Poisson and Compound Poisson processes. You should also explain, based on this realisation, why the (approximate) gradients for the Poisson and Compound Poisson processes that we simulated in the first second are the same.</b>

# Estimating the variance

You might be tempted to conclude at this stage that the Compound Poisson process is unecessary.  After all in the last two exercises we showed that the expectation, $\mathbb{E}[Y(t)]$, for a Compound Poisson process, $Y(t)$, with rate $\lambda$ and mean $\mu$ is the same as the expectation, $\mathbb{E}[N(t)]$, of a regular Poisson process, $N(t)$, with rate $\lambda\mu$.  Why then don't we model Compound Poisson Processes using a regular Poisson process with a suitably-scaled rate parameter?  Furthermore, (and returning to our original question) how can we determine that a process is a Compound Poisson process if we are just given information on how $Y(t)$ changes with time?  We will find an answer to these questions by considering the variance for the Poisson and Compound Poisson Processes.  In order to do this you will need to understand a little bit more about how the dynamic plotting tools work as we are going to use them to esimtate the variance for random variables rather than the means.  

Lets suppose that we have a call to the dynamic plotting tool that looks like this:

dynamicplot2( 1000, mean, compoundpoisson )

In essence this instruction tells python to generate a graph by calling the function mean 1000 times.  Furthermore, every time we call mean we are going to generate an image of the graph.  We can then merge all these images together to make a video in the same way that you can make a video by drawing pictures in a book and then quickly flipping through the pages.

If we look at the function mean we find that it looks something like this:

In [ ]:
def mean( ngen, myvar ) :
    cnt, mysum = 0., 0.
    while cnt < ngen :
        cnt += 1
        mysum += myvar()
        yield cnt, mysum / cnt

The function is passed two random arguments.  The first gives the number of random variables that should be generated.  The second is the function that is used to calculate the random variables.  To be clear this second argument is not a number it is a function, which we call within mean.  When we call dynamicplot2 using the compoundpoisson random variable generated this argument is the function that generates the compound poisson process.  By contrast when we call dynamicplot2 using the poisson random variable generate this is the function that generates the poisson random variable.  This is how we are able to use the same code to study multiple different random variables.

Lets now look at which the mean function does.  In the first line we initialise two real variables (cnt and mysum) to be equal to zero.  We then have a loop which runs until cnt=ngen.  Furthermore, in every iteration through this loop we add 1 to the variable cnt (cnt+=1).  We are thus going to do ngen iterations through this loop.  

The other thing that we do in the loop is that we call the random number generator (myvar) and add the random number that we get to the variable mysum.  mysum thus accumulates the sum of all the random variables that we have generated thus far.

The next bit is a bit complicated and I don't want you to get too bogged down in the details.  What you need to understand is that the line starting yield ensures that a new point is added to our graph at $x=$cnt and $y=$(mysum/cnt).  (If you are interested using the yield command ensures that this function is iterable so I can call dive in and out of the loop from another function elsewhere).  

With your renwed understanding of how mean works I would like to you to write a function that calculates how an estimate of the variance changes as you perform more experiments.  As you do this remember that the variance of a random variable, $X$, can be calculated using:

$$
\textrm{var}(X) = \mathbb{E}(X^2) - [\mathbb{E}(X)]^2
$$

I have written an outline of what you need to write below as well as code that will allow you to test whether or not you have written your code correctly.

In [ ]:
# You will need to modify this function
def variance( ngen, myvar ) :
    return 1
        
# You do not need to modify the code below  
# once you have implemented the variance correctly
# this will allow you to test what you have done
for data in mean(10,poisson) : 
    print( data )

Having written our function to estimate the variance lets use it to estimate variances for the compound poisson and poisson processes.  In the cell below I have written a call to dynamic plot that will allow you to estimate the variance for a poisson process with $\lambda=1$. 

In [ ]:
dynamicplot2( 1000, variance, poisson )

In the cell below I have written a call to dynamicplot2 that will allow you to estimate the variance for the Compound Poisson Process

In [ ]:
dynamicplot2( 1000, variance, compoundpoisson )

What are the estimates of the variance for the Poisson and Compound Poisson processes that you get from these simulations.  <b>Explain in your notes how you could use estimates of the mean and the variance like these to determine whether a random process is a Poisson process or a Compound Poisson Process.  In addition, explain how the rates of change for the expectation $\mathbb{E}[N(t)]$ and variance $\textrm{var}[N(t)]$ for a Poisson process $N(t)$ differ from those for a Compound Poisson Process $Y(t)$ and how you can use these differences to determine whether a random process is a Poisson process or a Compound Poisson Process  </b>